**This Notebook uses sample code from DDC to a single file which shows how to transform DWG files and Revit files to dataframes and then apply analytics (with 2050 Materials APIs)**


Import libraries and turn revit file into dataframe

In [6]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive  Shareddrives


In [7]:
import os, subprocess, pandas as pd, time
# Folder where Revit exporter is located
path_conv = "/content/drive/MyDrive/2050 Materials (New)/B_Research (NEW)/B_04_Projects/B_04_53_DataDrivenConstruction_Files/DDC_DWG_Converter/datadrivenlibs"
# The adress of the Revit file on computer
file_path = "/content/drive/MyDrive/2050 Materials (New)/B_Research (NEW)/B_04_Projects/B_04_53_DataDrivenConstruction_Files/RVT_projects/2022 rstadvancedsampleproject.rvt"

# Convert a Revit project to an open fotmat
process = subprocess.Popen([os.path.join(path_conv, "RvtExporter.exe"), file_path], cwd = path_conv)
process.wait() # waiting for process to be completed

# Read the converted Excel file
output_file = file_path[:-4] + "_rvt.xlsx"
df = pd.read_excel(output_file)
df.head(5)

PermissionError: [Errno 13] Permission denied: '/content/drive/MyDrive/2050 Materials (New)/B_Research (NEW)/B_04_Projects/B_04_53_DataDrivenConstruction_Files/DDC_DWG_Converter/datadrivenlibs/RvtExporter.exe'

Have a look at the Type name and volume of compoenents in the model

In [ ]:
df.columns = [col.split(' : ')[0] for col in df.columns]
QTO_group = df.groupby('Type Name')['Volume'].sum().reset_index().query("Volume != 0")
QTO_group

Save those as a bill of materials and plot in a bar chart

In [ ]:
# Save the filter data to a new Excel file
QTO_group.to_excel(r"D:\RVT_projects\QTO_group2_BoM.xlsx")

import matplotlib.pyplot as plt

plt.figure(figsize = (10,12))
plt.barh(QTO_group["Type Name"], QTO_group["Volume"], color = "skyblue")
plt.xlabel('Volume'); plt.ylabel('Type Name'); plt.title('Volume by Type Name'); plt.tight_layout()

Filter data by categories and plot

In [ ]:
# Filtering data in Revit and IFC projects

df[df['Category'].isin(['Wall', 'Window'])]
df.groupby(['Type']).sum().plot.pie(y='Length')
